In [1]:
%reload_ext autoreload
%autoreload 2

In [25]:
from types import SimpleNamespace
from typing import Any, Dict, List, Optional

from recon.dataset import Dataset
from pydantic import root_validator
from recon.types import Example, Span, Token
import numpy as np
from recon.augmentation import augment_example
from recon.operations import operation, registry

In [26]:
@operation("recon.v1.augment.create_kb_expansion", factory=True)
def kb_expansion(example: Example, *, spans_to_aliases_map: Dict[int, List[str]], n_augs: int = 1, sub_prob: float = 0.5):

    def augmentation(span: Span, *, spans_to_aliases_map: Dict[int, List[str]]):
        if hash(span) in spans_to_aliases_map:
            aliases = spans_to_aliases_map[hash(span)]

            if len(aliases) > 0:
                rand_alias = np.random.choice(aliases)
                index = aliases.index(rand_alias)
                del spans_to_aliases_map[hash(span)][index]
                return res

    return augment_example(example, augmentation, n_augs=n_augs, sub_prob=sub_prob, spans_to_aliases_map=spans_to_aliases_map)

In [27]:
factory = registry.operation_factories.get("recon.v1.augment.create_kb_expansion")

In [9]:
from spacy.kb import KnowledgeBase
from spacy.vocab import Vocab

vocab = Vocab().from_disk("/tmp/skills_extractor_v2/models/skills_linker/vocab/")
kb = KnowledgeBase(vocab)
kb.load_bulk("/tmp/skills_extractor_v2/models/skills_linker/kb")

In [29]:
operation = factory("test", [])

In [30]:
e = Example(text='This is text', spans=[])
ds = Dataset("test_ds", data=[e])
operation(ds)

TypeError: kb_expansion() missing 1 required keyword-only argument: 'spans_to_aliases_map'

In [ ]:
operation